In [ ]:
import pandas as pd
import random
from datetime import datetime, timedelta
import time
import re
import csv

In [ ]:
#Converting from tsv to csv bcz "rla_s.cpp" isn't working for *new* tsv added attribute values
#Change "sep" to '\t': If it's tsv File
df = pd.read_csv('/Users/dhruvrao/Desktop/Work/SequentialRLA/data/ds1.1.1', delimiter='\t', header=None, dtype=str)
df.to_csv('/Users/dhruvrao/Desktop/Work/SequentialRLA/data/updated_ds1.1.1', sep=',', index=False, header=False)

Adding Attributes namely Provider, BloodDate, BloodCost, XrayDate, XrayCost, DentalDate, DentalCost

In [ ]:
def random_hospital():
    return f"Hospital {random.randint(1, 20)}"

def random_date(start, end):
    random_days = random.randint(0, (end - start).days)
    return start + timedelta(days=random_days)

def extract_number(provider):
    match = re.search(r'\d+', provider)
    return int(match.group()) if match else 0

#Adding Attributes into **Initial Dataset**(SSN,LN,FN,DOD,DOB)
df1 = pd.read_csv('/Users/dhruvrao/Desktop/Work/SequentialRLA/data/updated_ds1.1.1', delimiter=',', header=None, dtype=str)
print(f"Initial Dataset shape is: {df1.shape}")
# df1.columns = ['SSN', 'LN', 'FN', 'DOD', 'DOB']
# df1.head()

#Adding Provider
df1[len(df1.columns)] = [f"Hospital {random.randint(1, 20)}" for _ in range(len(df1))]

#Blood Work Done
start_date = datetime(2018, 1, 1)
end_date = datetime(2020, 1, 1)
random_dates = [random_date(start_date, end_date).strftime('%m%d%Y') for _ in range(len(df1))]
df1[len(df1.columns)] = random_dates

#Defining Blood Cost
random_values = [random.randint(200, 500) for _ in range(len(df1))]
df1[len(df1.columns)] = random_values

#Xray Work Done
start_date = datetime(2018, 1, 1)
end_date = datetime(2020, 1, 1)
random_dates = [random_date(start_date, end_date).strftime('%m%d%Y') for _ in range(len(df1))]
df1[len(df1.columns)] = random_dates

#Defining Xray Work Cost
random_values = [random.randint(400, 600) for _ in range(len(df1))]
df1[len(df1.columns)] = random_values

#Dental Work Done
start_date = datetime(2018, 1, 1)
end_date = datetime(2020, 1, 1)
random_dates = [random_date(start_date, end_date).strftime('%m%d%Y') for _ in range(len(df1))]
df1[len(df1.columns)] = random_dates

#Defining Dental Cost
random_values = [random.randint(300, 500) for _ in range(len(df1))]
df1[len(df1.columns)] = random_values

#Writing Initial Dataset file with New Attributes namely (Provider,BD,BC,XD,XC,DD,DC)
df1.to_csv('/Users/dhruvrao/Desktop/Work/SequentialRLA/data/updated_ds1.1.1', sep=',', index=False, header=False)
# df1.columns = ['SSN', 'LN', 'FN', 'DOD', 'DOB', 'Provider', 'Blood Date', 'Blood Cost', 'X-Ray Date', 'X-Ray Cost', 'Dental Date', 'Dental Cost']
# df1.head()
print(f"Processing Dataset shape is: {df1.shape}")

Post Processing (Attributes and RLA Clustering)

Cost Analysis Process

In [ ]:
#Obatining Clustered(Indices) for newly generated attribute file from rlacpp(Indices in List of List)
start_time = time.time()
clusters = []
with open('/Users/dhruvrao/Desktop/Work/SequentialRLA/Out_recInds.csv', 'r') as f:
    for line in f:
        index_list = line.strip().split(',')
        valid_indexes = [int(index) for index in index_list if index.isdigit()]
    
        if valid_indexes:
            clusters.append(valid_indexes)

# print(clusters)
end_time = time.time()
work1 = end_time - start_time
print(f"Time taken for cluster making is: {end_time - start_time}")
print(f"The total length of cluster is: {len(clusters)}")

#Loading Main Dataset
main_df = pd.read_csv('/Users/dhruvrao/Desktop/Work/SequentialRLA/data/updated_ds1.1.1', header=None, dtype=str)

#Defining Columns in main Dataset
main_df.columns = [
    'SSN', 'LN', 'FN', 'DOD', 'DOB', 
    'Provider', 
    'BloodDate', 'BloodCost', 
    'XrayDate', 'XrayCost', 
    'DentalDate', 'DentalCost'
]

#Format Change for caluclating threshold meetings
main_df['BloodDate'] = pd.to_datetime(main_df['BloodDate'], format='%m%d%Y')
main_df['XrayDate'] = pd.to_datetime(main_df['XrayDate'], format='%m%d%Y')
main_df['DentalDate'] = pd.to_datetime(main_df['DentalDate'], format='%m%d%Y')
#Adding Cost Columns
main_df['BloodCostSaving'] = 0
main_df['XrayCostSaving'] = 0
main_df['DentalCostSaving'] = 0

total_blood_cost_sum = 0
total_xray_cost_sum = 0
total_dental_cost_sum = 0
    
start_time = time.time()
#Work Sorting wrt Date and Calculating Cost
output_file = "/Users/dhruvrao/Desktop/Work/SequentialRLA/data/cost_updated_ds1.1.1"
with open(output_file, "w") as f:
    for i, cluster in enumerate(clusters):
        cluster_df = main_df.iloc[cluster]
        
        sorted_blood_df = cluster_df.sort_values(by=['BloodDate'])
        for j in range(len(sorted_blood_df) - 1):
            current_row = sorted_blood_df.iloc[j]
            next_row = sorted_blood_df.iloc[j + 1]

            if (next_row['BloodDate'] - current_row['BloodDate']).days <= 7:
                sorted_blood_df.at[sorted_blood_df.index[j], 'BloodCostSaving'] = next_row['BloodCost']

        main_df.loc[sorted_blood_df.index, 'BloodCostSaving'] = sorted_blood_df['BloodCostSaving']

        sorted_xray_df = cluster_df.sort_values(by=['XrayDate'])
        for j in range(len(sorted_xray_df) - 1):
            current_row = sorted_xray_df.iloc[j]
            next_row = sorted_xray_df.iloc[j + 1]

            if (next_row['XrayDate'] - current_row['XrayDate']).days <= 150:
                sorted_xray_df.at[sorted_xray_df.index[j], 'XrayCostSaving'] = next_row['XrayCost']

        main_df.loc[sorted_xray_df.index, 'XrayCostSaving'] = sorted_xray_df['XrayCostSaving']
        
        sorted_dental_df = cluster_df.sort_values(by=['DentalDate'])
        for j in range(len(sorted_dental_df) - 1):
            current_row = sorted_dental_df.iloc[j]
            next_row = sorted_dental_df.iloc[j + 1]

            if (next_row['DentalDate'] - current_row['DentalDate']).days <= 180:
                sorted_dental_df.at[sorted_dental_df.index[j], 'DentalCostSaving'] = next_row['DentalCost']

        main_df.loc[sorted_dental_df.index, 'DentalCostSaving'] = sorted_dental_df['DentalCostSaving']
        
        #Changing Back Original Date Format
#         sorted_dental_df['BloodDate'] = sorted_dental_df['BloodDate'].dt.strftime('%m%d%Y')
#         sorted_dental_df['XrayDate'] = sorted_dental_df['XrayDate'].dt.strftime('%m%d%Y')
#         sorted_dental_df['DentalDate'] = sorted_dental_df['DentalDate'].dt.strftime('%m%d%Y')
        
        #Changing CostSaving Values to Numeric
        sorted_blood_df['BloodCostSaving'] = pd.to_numeric(sorted_blood_df['BloodCostSaving'], errors='coerce')
        sorted_xray_df['XrayCostSaving'] = pd.to_numeric(sorted_xray_df['XrayCostSaving'], errors='coerce')
        sorted_dental_df['DentalCostSaving'] = pd.to_numeric(sorted_dental_df['DentalCostSaving'], errors='coerce')
        
        #Calculating Total Cost Savings
        blood_cost_sum = sorted_blood_df['BloodCostSaving'].sum()
        xray_cost_sum = sorted_xray_df['XrayCostSaving'].sum()
        dental_cost_sum = sorted_dental_df['DentalCostSaving'].sum()
        
#         #Sorting w.r.t Provider Number for Visulaization
#         sorted_cluster_df['ProviderNumber'] = sorted_cluster_df['Provider'].apply(extract_number)
#         sorted_cluster_df = sorted_cluster_df.sort_values(by=['ProviderNumber'], ascending=True)
#         sorted_cluster_df = sorted_cluster_df.drop(columns=['ProviderNumber'])
        
        f.write(f"Cluster {i+1} sorted records:\n")
#         f.write(sorted_cluster_df.to_string())
#         f.write("\n")
        f.write(f"\nTotal BloodCost Savings: {blood_cost_sum}, Total XrayCost Savings: {xray_cost_sum}, Total DentalCost Savings: {dental_cost_sum}\n")
        f.write("\n\n")

        total_blood_cost_sum += blood_cost_sum
        total_xray_cost_sum += xray_cost_sum
        total_dental_cost_sum += dental_cost_sum

end_time = time.time()
work2 = end_time - start_time
print(f"Total Blood Cost saved = {total_blood_cost_sum}")
print(f"Total Xray Cost saved = {total_xray_cost_sum}")
print(f"Total Dental Cost saved = {total_dental_cost_sum}")
print(f"Time taken for Analyzing all Cost Work is: {end_time-start_time} seconds")
print(f"Analysis Completed, Results in: {output_file} file")
print(f"Total time taken for the process is:{work1+work2}")